## Assignment 4: Obsevational Studies and Applied ML

### Deadline
November 21st,11:59PM

### Important notes

Make sure you push on GitHub your notebook with all the cells already evaluated. Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you implemented. Back up any hypotheses and claims with data, since this is an important aspect of the course. Please write all your comments in English, and use meaningful variable names in your code. Your repo should have a single notebook (plus the data files necessary) in the master branch. If there are multiple notebooks present, we will not grade anything.

Use this legendary link to create your repository: [link](https://classroom.github.com/g/YXtsr0QK)

In [1]:
# Imports
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_folder = './data/'

## Task 1: Boosting the economy by incentivizing self-employment

Assume the biggest priority of the local government in 2018 is to increase per-capita income. To do so, the officials plan to adopt a strategy for incentivizing self-employment through a series of campaigns, educational programs, and dedicated funds.

Since it is unethical and impossible in this setting to run a controlled experiment involving citizens (e.g., fire employees and force them to self-employ), the officials have asked you, the data scientist, to establish the effect of self-employment on the economy, relying on observational data.

**A)** You will be working with the full US 2015 census dataset (acs2015_county_data.csv, available at https://www.kaggle.com/muonneutrino/us-census-demographic-data#acs2015_county_data.csv). Using suitable methods, determine and quantify the dependency between the percentage of self-employed citizens and per capita income across all 3,212 US counties. Do citizens in counties that have a higher percentage of self-employed people earn more per capita?

**B)** The pilot program will involve all counties within a limited set of three US states. Set A includes Wisconsin, Tennessee, and  Minnesota. Quantify the dependency of per-capita income on self-employment rates across all the counties in set A.

**C)** In which state within set A is the observed effect of self-employment on per-capita income the strongest?

**D)** Set B includes New Jersey, Kansas, and Rhode Island. Repeat the analysis from steps B and C above, but now for set B. In which of the two sets A and B (if any) would you recommend incentivizing self-employment? Explain your reasoning.

Hint: It is useful to add a notion of confidence to your results and explore the data visually. You are allowed to use the SciPy library.

## Task 2: All you need is love… And a dog!

Here we are going to build a classifier to predict whether an animal from an animal shelter will be adopted or not (aac_intakes_outcomes.csv, available at: https://www.kaggle.com/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes/version/1#aac_intakes_outcomes.csv). You will be working with the following features:

1. *animal_type:* Type of animal. May be one of 'cat', 'dog', 'bird', etc.
2. *intake_year:* Year of intake
3. *intake_condition:* The intake condition of the animal. Can be one of 'normal', 'injured', 'sick', etc.
4. *intake_number:* The intake number denoting the number of occurrences the animal has been brought into the shelter. Values higher than 1 indicate the animal has been taken into the shelter on more than one occasion.
5. *intake_type:* The type of intake, for example, 'stray', 'owner surrender', etc.
6. *sex_upon_intake:* The gender of the animal and if it has been spayed or neutered at the time of intake
7. *age_upon\_intake_(years):* The age of the animal upon intake represented in years
8. *time_in_shelter_days:* Numeric value denoting the number of days the animal remained at the shelter from intake to outcome.
9. *sex_upon_outcome:* The gender of the animal and if it has been spayed or neutered at time of outcome
10. *age_upon\_outcome_(years):* The age of the animal upon outcome represented in years
11. *outcome_type:* The outcome type. Can be one of ‘adopted’, ‘transferred’, etc.

**A)** Load the dataset and convert categorical features to a suitable numerical representation (use dummy-variable encoding). Split the data into a training set (80%) and a test set (20%). Pair each feature vector with the corresponding label, i.e., whether the outcome_type is adoption or not. Standardize the values of each feature in the data to have mean 0 and variance 1. The use of external libraries is not permitted in part A, except for numpy and pandas.

**B)** Train a logistic regression classifier on your training set. Logistic regression returns probabilities as predictions, so in order to arrive at a binary prediction, you need to put a threshold on the predicted probabilities. For the decision threshold of 0.5, present the performance of your classifier on the test set by displaying the confusion matrix. Based on the confusion matrix, manually calculate accuracy, precision, recall, and F1-score with respect to the positive and the negative class. Vary the value of the threshold in the range from 0 to 1 and visualize the value of accuracy, precision, recall, and F1-score (with respect to both classes) as a function of the threshold. The shelter has a limited capacity and has no other option but to put to sleep animals with a low probability of adoption. What metric (precision, recall, accuracy, or F1-score) and with respect to what class is the most relevant when choosing the threshold in this scenario, and why? Explain your reasoning.

**C)** Reduce the number of features by selecting the subset of the k best features. Use greedy backward selection to iteratively remove features. Evaluate performance and visualize the result using 5-fold cross-validation on the training set as a function of k, where k = 1, 5, 10, 15, 20, 25, 30. Choose the optimal k and justify your choice. Interpret the top-k features and their impact on the probability of adoption.

**D)** Train a random forest. Use 5-fold cross-validation on the training set to fine-tune the parameters of the classifier using a grid search on the number of estimators "n_estimators" and the max depth of the trees "max_depth". For the chosen parameters, estimate the performance of your classifier on the test set by presenting the confusion matrix, accuracy, precision, recall, and F1-score with respect to both classes and compare the performance with the performance of the logistic regression. Interpret the results.

You are allowed to use the scikit-learn library to implement your classifiers.

### TASK 2A
**A)** Load the dataset and convert categorical features to a suitable numerical representation (use dummy-variable encoding). Split the data into a training set (80%) and a test set (20%). Pair each feature vector with the corresponding label, i.e., whether the outcome_type is adoption or not. Standardize the values of each feature in the data to have mean 0 and variance 1. The use of external libraries is not permitted in part A, except for numpy and pandas.

In [3]:
# Loading the dataset
aac = pd.read_csv(data_folder + 'aac_intakes_outcomes.csv.zip', compression = 'zip')

# Only keep relevant columns
aac = aac[['animal_type','intake_year','intake_condition','intake_number','intake_type','sex_upon_intake',
          'age_upon_intake_(years)','time_in_shelter_days','sex_upon_outcome','age_upon_outcome_(years)',
          'outcome_type']]

In [4]:
aac.head()

,animal_type,intake_year,intake_condition,intake_number,intake_type,sex_upon_intake,age_upon_intake_(years),time_in_shelter_days,sex_upon_outcome,age_upon_outcome_(years),outcome_type
0,Dog,2017,Normal,1.0,Stray,Neutered Male,10.0,0.588194,Neutered Male,10.0,Return to Owner
1,Dog,2014,Normal,2.0,Public Assist,Neutered Male,7.0,1.259722,Neutered Male,7.0,Return to Owner
2,Dog,2014,Normal,3.0,Public Assist,Neutered Male,6.0,1.113889,Neutered Male,6.0,Return to Owner
3,Dog,2014,Normal,1.0,Owner Surrender,Neutered Male,10.0,4.970139,Neutered Male,10.0,Transfer
4,Dog,2013,Injured,1.0,Public Assist,Neutered Male,16.0,0.119444,Neutered Male,16.0,Return to Owner


In [5]:
# Define features x and outcome y
x = aac.drop(axis=1, labels='outcome_type')
y = aac[['outcome_type']]

# Outcome to adoption (1) or not (0)
y_adopt = y[y['outcome_type'] == 'Adoption']
y_not_adopt = y[y['outcome_type'] != 'Adoption']
y_adopt['Outcome'], y_not_adopt['Outcome'] = 1, 0
y_adopt, y_not_adopt = y_adopt[['Outcome']], y_not_adopt[['Outcome']]

y = pd.concat([y_adopt,y_not_adopt])
y.sort_index(axis=0,ascending=True, inplace = True);

# Dummy variables encoding
x = pd.get_dummies(x)

In [6]:
print(y.shape, x.shape)

(79672, 1) (79672, 32)


In [7]:
def split_data(x, y, ratio, myseed=1):
    """ Split the dataset in function of the split ratio """
    
    # The data should be splited randomly in order to avoid any perturbation which
    # could be caused by the original order of the data
    np.random.seed(myseed)
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_train, index_test = indices[: index_split], indices[index_split:]
    
    # Create to splitted sets (train and test)
    x_train, x_test = x.iloc[index_train], x.iloc[index_test]
    y_train, y_test = y.iloc[index_train], y.iloc[index_test]

    return x_train, x_test, y_train, y_test

x_tr, x_te, y_tr, y_te = split_data(x, y, 0.8)

In [8]:
# Standardize
mean_tr = np.mean(x_tr, axis=0)
std_tr = np.std(x_tr, axis=0)
x_tr = (x_tr - mean_tr) / std_tr
x_te = (x_te - mean_tr) / std_tr

In [9]:
x_tr.head()

,intake_year,intake_number,age_upon_intake_(years),time_in_shelter_days,age_upon_outcome_(years),animal_type_Bird,animal_type_Cat,animal_type_Dog,animal_type_Other,intake_condition_Aged,...,sex_upon_intake_Intact Female,sex_upon_intake_Intact Male,sex_upon_intake_Neutered Male,sex_upon_intake_Spayed Female,sex_upon_intake_Unknown,sex_upon_outcome_Intact Female,sex_upon_outcome_Intact Male,sex_upon_outcome_Neutered Male,sex_upon_outcome_Spayed Female,sex_upon_outcome_Unknown
63044,1.199993,-0.276527,-0.036183,-0.402772,-0.048219,-0.065948,-0.768248,-1.148099,4.109571,-0.062878,...,-0.649349,-0.683996,-0.435801,-0.402206,3.264082,-0.361039,-0.373802,-0.743085,-0.686815,3.264082
48857,0.433106,-0.276527,-0.382544,-0.236281,-0.393704,-0.065948,1.301663,-1.148099,-0.243334,-0.062878,...,-0.649349,1.461997,-0.435801,-0.402206,-0.306365,-0.361039,-0.373802,1.345740,-0.686815,-0.306365
77854,1.966880,-0.276527,-0.382544,-0.402488,-0.393704,-0.065948,-0.768248,-1.148099,4.109571,-0.062878,...,-0.649349,-0.683996,-0.435801,-0.402206,3.264082,-0.361039,-0.373802,-0.743085,-0.686815,3.264082
65109,1.199993,-0.276527,-0.724159,-0.396979,-0.734458,-0.065948,-0.768248,0.871005,-0.243334,-0.062878,...,-0.649349,1.461997,-0.435801,-0.402206,-0.306365,-0.361039,2.675214,-0.743085,-0.686815,-0.306365
19219,-1.100668,-0.276527,1.002897,0.145371,0.988239,-0.065948,-0.768248,0.871005,-0.243334,-0.062878,...,-0.649349,-0.683996,2.294626,-0.402206,-0.306365,-0.361039,-0.373802,1.345740,-0.686815,-0.306365


### TASK 2B

**B)** Train a logistic regression classifier on your training set. Logistic regression returns probabilities as predictions, so in order to arrive at a binary prediction, you need to put a threshold on the predicted probabilities. For the decision threshold of 0.5, present the performance of your classifier on the test set by displaying the confusion matrix. Based on the confusion matrix, manually calculate accuracy, precision, recall, and F1-score with respect to the positive and the negative class. Vary the value of the threshold in the range from 0 to 1 and visualize the value of accuracy, precision, recall, and F1-score (with respect to both classes) as a function of the threshold. The shelter has a limited capacity and has no other option but to put to sleep animals with a low probability of adoption. What metric (precision, recall, accuracy, or F1-score) and with respect to what class is the most relevant when choosing the threshold in this scenario, and why? Explain your reasoning.

In [10]:
logistic = LogisticRegression(solver='lbfgs')

In [11]:
logistic.fit(x_tr, y_tr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
def predict_labels(pred, threshold = 0.5):
    """ Dichotomic classification of prediction, given a specific threshold and raw probabilities. """
    pred[np.where(pred <= threshold)] = 0
    pred[np.where(pred > threshold)] = 1
    
    return pred

pred = predict_labels(logistic.predict_proba(x_te))
pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [76]:
def confusion_matrix(pred, y):
    """ Compare pred and y in order to determine the values of the confusion matrix """
    pd.DataFrame({'Prediction':pred[:,1]})
    df = df_pred.join(y)
    tp = len((df[df.Prediction == 1])[df.Outcome == 1])
    tn = len((df[df.Prediction == 0])[df.Outcome == 0])
    fp = len((df[df.Prediction == 1])[df.Outcome == 0])
    fn = len((df[df.Prediction == 0])[df.Outcome == 1])
    
    table = {'Pred = 0': {'y = 1' : fn, 'y = 0' : tn}, 'Pred = 1' : {'y = 1' : tp, 'y = 0' : fp} }
    df_table = pd.DataFrame.from_dict(table)
    
    return df_table
    
confusion_matrix(pred, y_te).head()

,Pred = 0,Pred = 1
y = 0,1104,939
y = 1,622,533
